# 0. Import

In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import similarity_metric as sm
import formality_metric as fm

In [3]:
def switch_from_string_to_integer(mungchi_string):
    # 슬래시와 공백을 제외한 글자 수를 계산
    parts = mungchi_string.split('/')  # 슬래시를 기준으로 문자열을 나눔
    mungchi_integer = [len(part.strip()) for part in parts]  # 각 부분을 공백 제거 후 길이 계산
    return mungchi_integer

# 1. Load Dataset

In [4]:
dataframe = pd.read_json('./../data/dataset_v2.0.json')
print(dataframe.columns)
print(dataframe.shape)

Index(['title', 'lyrics', 'genre', 'check_only_korean', 'line_samples',
       'verse_samples', 'total_samples', 'line_sample_word_mungchi_string',
       'line_sample_word_mungchi_integer', 'line_sample_line_mungchi_string',
       'line_sample_line_mungchi_integer', 'verse_sample_word_mungchi_string',
       'verse_sample_word_mungchi_integer', 'verse_sample_line_mungchi_string',
       'verse_sample_line_mungchi_integer', 'total_sample_word_mungchi_string',
       'total_sample_word_mungchi_integer', 'total_sample_line_mungchi_string',
       'total_sample_line_mungchi_integer'],
      dtype='object')
(6090, 19)


In [5]:
df = dataframe.copy()

# 2. Inference
- 가상의 테스트셋에 해당하는 노래 10곡
- 여기서 input으로 요청하는 음절수가 golden_mungchi_integer에 저장되어야 함.

In [6]:
temp_test_df = df.loc[df["check_only_korean"]==True].sample(n=10, random_state=42)
temp_test_df.shape

(10, 19)

# 3. Evaluation

In [7]:
# 의미 유사도 산출을 위한 encoder 불러오기
model = AutoModel.from_pretrained("kakaobank/kf-deberta-base")
tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
predict_mungchi_integer_list = []
semantic_sim_list = []
lexical_sim_list = []
acc_form_list = []
mse_form_list = []
our_form_list = []

for row in range(len(temp_test_df)):
    
    golden_lyrics = temp_test_df.iloc[row]['lyrics']
    
    # golden_mungchi_integer = 실제 input으로 들어가는 음절 수
    # type : list [2,2,2]
    golden_mungchi_integer = temp_test_df.iloc[row]['line_sample_word_mungchi_integer'][0]
    # golden_mungchi_string = 생성된 가사와 비교해볼 string -> 실제로 사용은 안됨.
    # type : 뭉치가 ' / '로 구분된 하나의 str ('내가 / 만든 / 가사')
    golden_mungchi_string = temp_test_df.iloc[row]['line_sample_word_mungchi_string'][0]
    # predict_mungchi_string = 동일 주제, 동일 장르로 생성된 string
    # type : 뭉치가 ' / '로 구분된 하나의 str ('내가 / 만든 / 가사')
    predict_mungchi_string = temp_test_df.iloc[row]['line_sample_word_mungchi_string'][0]
    
    predict_mungchi_integer = switch_from_string_to_integer(predict_mungchi_string)
    predict_mungchi_integer_list.append(predict_mungchi_integer)
    
    # check log
    # print(f'<golden_lyrics>\n{golden_lyrics}\n')
    # print(f'<predict_mungchi>\n{predict_mungchi_string}')
    
    # evaluate test data
    semantic_sim = sm.eval_semantic_sim(model, tokenizer, golden_lyrics, predict_mungchi_string)
    lexical_sim = sm.eval_lexical_sim_bleu(golden_lyrics, predict_mungchi_string)
    acc_form, mse_form = fm.eval_form(golden_mungchi_integer, predict_mungchi_integer)
    our_form = fm.eval_our_form(golden_mungchi_integer, predict_mungchi_integer)
    
    # save scores
    semantic_sim_list.append(semantic_sim)
    lexical_sim_list.append(lexical_sim)
    acc_form_list.append(acc_form)
    mse_form_list.append(mse_form)
    our_form_list.append(our_form)

eval_df = pd.DataFrame({'original_lyrics' : temp_test_df['lyrics'],
                        'input_mungchi_integer' : temp_test_df['line_sample_word_mungchi_integer'],
                        'input_mungchi_string' : temp_test_df['line_sample_word_mungchi_string'],
                        'generated_mungchi_string' : temp_test_df['line_sample_word_mungchi_string'],
                        'generated_mungchi_integer' : predict_mungchi_integer_list,
                        'semantic_sim' : semantic_sim_list,
                        'bleu_lexical_sim' : lexical_sim_list,
                        'acc_form' : acc_form_list,
                        'mse_form' : mse_form_list,
                        'our_form' : our_form_list})

# save evaluation result
eval_df.to_csv('evalutation_result.csv', index=False, encoding='utf-8-sig')

<golden_lyrics>
거리에 오 겹쳐진 그녀
모습 속에는
오 난 그어떤 그리움도
찾아볼 순 없군요
거리에 일렁이는 그녀
모습 속에는
오 난 그어떤 외로움도
찾아볼 순 없군요
처음엔 사랑이란 게 참 쉽게
영원할 거라
그렇게 믿었었는데 그렇게
믿었었는데
나에게 사랑이란 게 또다시
올수 있다면
그때는 가깝진 않게 그다지
멀지도 않게
난 예
벤치에 앉아있는 그녀
모습 속에는
오 난 그어떤 그리움도 찾아볼 순
없군요
벤치에 들려오는 그녀
웃음 속에는
오 난 그어떤 외로움도
찾아볼 순 없군요
처음엔 사랑이란 게 참 쉽게
영원할 거라
그렇게 믿었었는데 그렇게
믿었었는데
나에게 사랑이란 게 또다시
올수 있다면
그때는 가깝진 않게 그다지
멀지도 않게
머린 아픈데 오 너는 없고
그때 또 차오르는 니 생각에
어쩔 수 없는 나의 맘 그때의 밤
나에겐 사랑이란 게 아 사랑이란
처음엔 사랑이란 게 참 쉽게
영원할 거라
그렇게 믿었었는데 그렇게
믿었었는데
나에게 사랑이란 게
라랄라라 워 허허어 허어 워
허어허어 예
라랄라라 워 허허어 허어 워
허어허어 예

<predict_mungchi>
거리에 / 오 / 겹쳐진 / 그녀
<golden_lyrics>
참 많이 울었죠
그대 마음 다 알아요
어제도 그대 울 생각에
많이 염려 했어요
더 울게 될 거예요
그대에겐 아직
더 많은 만남들이 있다는 걸
그대 알아두기를
꼭 당신이 아니더라도
더 좋은 누군가라도
흐르는 강물에 지나지 않아요
하지만 우리는 평생
흐르는 강물을
붙잡으며 살아갈 거예요
이해할 수 없는 그대
그대도 나를 절대 이해할 수 없죠
많이 힘들겠죠
나도 알고 있어요
하지만 지금 우리 사이
받아들여야해요
조금 더 만난다 해도
우린 그대롤거예요
그러니 그대 두려워 마요
괜찮아질 거야
혹 당신이 아니더라도
더 좋은 누군가라도
흐르는 강물에 지나지 않아요
하지만 우리는 평생
흐르는 강물을
붙잡으며 살아갈 거예요
이해할 수 없는 그대
그대도 나를 절대 이해할 수 없죠
혹 당신이 이해한대도
세상을 버린다 해도
시간이 지나면 알거라 믿어요
그

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


<golden_lyrics>
거리를 나 혼자 걸었네
내게는 아무도 없었네
차가운 바람 불때면
내 마음 왠지 쓸쓸해지네
조금씩 비가 내리네
어둠은 갈수록 짙어가네
빗속을 혼자 걷는 이 마음
그대는 아는지 흥 모르는지
아 이 비 그치면
그대 찾아 봐야지
아무리 험한 산 일지라도
난 그대를 잊을 수 없어
아무리 미운 너였지만은
난 아직도 널 사랑해
아 이 비 그치면
그대 찾아 봐야지
아무리 험한 산 일지라도
난 그대를 잊을 수 없어
아무리 미운 너였지만은
난 아직도 널 사랑해
난 아직도 널 사랑해
난 아직도 널 사랑해

<predict_mungchi>
거리를 / 나 / 혼자 / 걸었네
<golden_lyrics>
당신은 시계처럼 정확한 분이지만
난 정말 당신처럼 정확할순 없어요
원하는 사랑을 드릴수도 있지만
내마음 시계처럼 맞춰놓진 말아요
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
답답해서 싫어요 답답해서 싫어
답답해서 싫어요 답답해서 싫어
당신은 시계처럼 정확한 분이지만
난 정말 당신처럼 정확할순 없어요
원하는 사랑을 드릴수도 있지만
내마음 시계처럼 맞춰놓진 말아요
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
답답해서 싫어요 답답해서 싫어
답답해서 싫어요 답답해서 싫어
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
사랑을 시간으로 따질수는 없잖아
새장속의 새처럼 답답해서 싫어요
답답해서 싫어요 답답해서 싫어
답답해서 싫어요 답답해서 싫어
답답해서 싫어요 답답해서 싫어
답답해서 싫어요 답답해서 싫어

<predict_mungchi>
당신은 / 시계처럼 / 정확한 / 분이지만
<golden_lyrics>
쉬는 날이라
집에만 있고 싶지가 않아서
오랜만이야
모처럼 나갈 준비를 하고
행복해 보이는 사람들 속에
웃고 떠들면서
그렇게 너를 조금씩
잊어 가나 봐
널 보낸 길에 서서 참 많이 울

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


<golden_lyrics>
가지려고 가지려고
가져보려고
무던히
원하고 바랬죠
잠시라도 그대곁에
있는 동안엔
모른척
내것이라 믿었죠
웃는 그대 얼굴을
한참 못본 후에야
알았죠
더는 어려운 일인 걸
갖지 못한
그대 마음이
못내 서러웠지만
보내야 했죠
사랑이란 못된 이유로
그대 맘을 잡기엔
너무 늦어버린 걸
알죠
잊으려고 잊으려고
잊어보려고
여전히
입술을 깨물죠
하루라도 그대없이
살아보려고
아닌 척
웃어보기도 하죠
어떡하죠 내 맘이
내 맘같지 않아서
눈이 시리도록
보고 싶은데
바보같은
나는 이래요
사랑했던 순간만
기억해내고
내 마음도
그대 마음도
모두 잃어버린 채
아파하네요
눈물도
기다려보겠단 말도
보일 수 없던
나를 모르죠
변해버린 건 그댄데
왜 내가 미안할까요
갖지 못한
그대 마음이
못내 서러웠지만
보내야죠
사랑이란 못된 이유로
그대 맘을 잡기엔
너무 늦어버린 걸
알죠

<predict_mungchi>
가지려고 / 가지려고


...

<golden_lyrics>
어떤 약속도 없는 그런 날에
너만 혼자 집에 있을때
넌 옛 생각이 나는지
그럴땐 어떡하는지
또 우울한 어떤날
음 비 마저 내리고
늘 우리가 듣던 노래가
라디오에서 나오면
나처럼 울고 싶은지
왜 자꾸만 후회되는지
나의 잘못했던 일과
너의 따뜻한 마음만
더 생각나
너의 방안을 정리하다가
내 사진이
혹시 나오면
넌 그냥 찢고 마는지
한참을 바라보는지
또 우울한 어떤날
음 비 마저 내리고
늘 우리가 듣던 노래가
라디오에서 나오면
나처럼 울고 싶은지
왜 자꾸만 후회되는지
나의 잘못했던 일과
너의 따뜻한 마음만
더욱 생각나
그대여 나와 같다면
내 마음과 똑같다면
그냥 나에게 오면 돼
널 위해 비워둔 내 맘
그 자리로
그대여 나와 같다면
내 마음과 똑같다면
그냥 나에게 오면 돼
널 위해 비워둔 내 맘
그 자리로
나처럼 울고 싶은지
왜 자꾸만 후회되는지

<predict_mungchi>
어떤 / 약속도 / 없는 / 그런 / 날에
<golden_lyrics>
희미해지는
지난 추억 속의 그 길을
이젠 다시 걸어볼 순 없다 하여도
이내 가슴에 지워버릴 수 없는
그때 그 모든 기억들
그대의 사랑이 지나가는 자리엔
홀로된 나의 슬픔 고독 뿐
그때가 다시 올 순 없어도
지나간 추억만은 영원히
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 사랑 뿐
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 기억 뿐
희미해지는
지난 추억 속의 그 길을
이젠 다시 걸어볼 순 없다 하여도
이내 가슴에 지워버릴 수 없는
그때 그 모든 기억들
그대의 사랑이 지나가는 자리엔
홀로된 나의 슬픔 고독 뿐
그때가 다시 올 순 없어도
지나간 추억만은 영원히
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 사랑 뿐
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 기억 뿐
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 사랑 뿐
다시 돌아올 수 없는 사랑에
홀로 돌이켜본 추억은
다만 아름답던 기억 뿐
다시 돌

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

<golden_lyrics>
왜 몰랐을까
항상 나를 위했던
천사처럼 착했던 널
왜 몰랐을까
까다로웠던 나를
말없이 다 받아준 널
헤어지잔 말
진심이 아니었는데
뒤늦은 후회가 슬퍼
고마운 줄 모르고
너를 실망시켰던
지난날의 내가 미워
내 철없음을 용서해
난 사랑을 잘 몰랐어
널 잊으라고 하지마
못난 심장이 매일 울어
너 돌아오면 안 되니
너 없으니까 죽을 것 같아
이젠 알겠어
내겐 너무 소중한 너야
덜컥 겁이나
이대로 널 잃을까봐
두려워 조바심이 나
받지 않는 전화를
수십 번도 더 걸어
내가 정말 미쳤나봐
내 철없음을 용서해
난 사랑을 잘 몰랐어
널 잊으라고 하지마
못난 심장이 매일 울어
너 돌아오면 안 되니
너 없으니까 죽을 것 같아
이젠 알겠어
내겐 너무 소중한 너야
널 기다리고 있을게
그 자리에 서 있을게
널 힘들게 해 미안해
내 잘못들도 모두 잊어
나 정말 많이 울었어
멍든 가슴이 터질 것 같아
제발 돌아와
우리 다신 헤어지지마
너 없으니까
못 보니까 죽을 것 같아

<predict_mungchi>
왜 / 몰랐을까


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


<golden_lyrics>
만약에 말야 우리 조금
어렸었다면 지금 어땠었을까
만약에 말야 우리 지금 이럴
거라면 후회 하진 않을까
세월에 닫혀진 우리의 연이
더는 허락되어 지지 않아도
만약에 말야 우리 조금
어렸었다면 지금 어땠었을까
만약에 말야 우리 지금
이럴 거라면 후회 하진 않을까
세상에 닫혀진 우리의 날이
이젠 바랜 기억 뿐 일 지라도
만약에 말야 우리 조금
어렸었다면 지금 어땠었을까
만약에 말야 우리 지금
이럴 거라면 후회 하진 않을까
만약에 말야 만약에 말야
만약에 말야 우리 같은
마음이라면 다시 되돌아볼까
만약에 말야 우리 정말
사랑했다면 지워 낼 수 있을까
만약에 말야

<predict_mungchi>
만약에 / 말야 / 우리 / 조금
